In [17]:
import os
import sys
sys.path.append('/zfsauton3/home/bparia/hf_moe/')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
from tabulate import tabulate

In [83]:
with open('../data/m5/data.pkl', 'rb') as fin:
    tree, ts_data, \
        (global_cont_feats, global_cat_feats, global_cat_dims) \
            = pickle.load(fin)

leaf_mat = tree.leaf_matrix
num_leaf = np.sum(leaf_mat, axis=1, keepdims=True)
cmat = leaf_mat / num_leaf

In [88]:
def get_cerror(path):
    levels = tree.levels
    with open(path, 'rb') as fin:
        pred, _ = pickle.load(fin)
    
    cpred = pred @ cmat.T

    errs = []
    for d in levels:
        sub_pred = pred[:, levels[d]]
        sub_cpred = cpred[:, levels[d]]
        diff = np.mean(np.abs(sub_cpred - sub_pred)) / np.mean(np.abs(sub_cpred))
        errs.append(diff)
    
    return errs

def get_average(model):
    errs = []
    for i in range(1, 11):
        path = os.path.join('../logs/', model, f'run_{i}', 'test.pkl')
        err = get_cerror(path)
        errs.append(err)
    
    errs = np.asarray(errs)
    errs = np.mean(errs, axis=0)
    print(errs)

In [82]:
get_average('favorita/hid_20_exp_8_reg_norm/')
get_average('favorita/hid_20_exp_8_norm/')
get_average('favorita/hid_64_exp_1_norm/')

[0.0355816  0.03990585 0.04061769 0.        ]
[0.03834413 0.04409121 0.04534195 0.        ]
[0.04305472 0.04473148 0.04209769 0.        ]


In [70]:
get_average('tourism/hid_10_exp_4_reg_norm/')
get_average('tourism/hid_30_exp_1_norm/')


[0.10895054 0.10223943 0.08093576 0.07677878 0.        ]
[0.08818724 0.08239052 0.07711098 0.07855723 0.        ]


In [89]:
get_average('m5/hid_20_exp_8_reg_norm/')
get_average('m5/hid_20_exp_8_norm/')
get_average('m5/hid_64_exp_1_norm/')

[0.02489813 0.02959804 0.03232974 0.        ]
[0.03216533 0.03302003 0.03686976 0.        ]
[0.04243737 0.05721546 0.04739717 0.        ]
